In [4]:
from XMTR import MTR
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [3]:
my_array = [i for i in range(100, 1001, 20)]
len(my_array)

46

In [9]:
def calc_al_error(instance, perc):
  # the error should be non zero
  return (abs(instance)+0.1)*perc 


# load data
slump_data = arff.loadarff('slump.arff')
slump_df = pd.DataFrame(slump_data[0])

# get column names
column_names = slump_df.columns

# get data/target values
f_n = column_names[:7]
t_n = column_names[7:]

X = slump_df[f_n]
y = slump_df[t_n]

# convert to numpy
X = X.to_numpy()
y = y.to_numpy()

# scale target values
scaler = MinMaxScaler(feature_range=(0,10))
y = scaler.fit_transform(y)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
allowed_error = 0.1

max_ds = [2]
min_s_ls = [10]
m_fs = [None]

ests = [i for i in range(100, 1001, 50)]
rule_lengths = []
total = (len(max_ds)*len(min_s_ls)*len(m_fs)*len(ests))
counter = 1
min_len = (len(f_n)+1) # just a ref, should be > than len(feature_names)
for max_d in max_ds:
  for min_s_l in min_s_ls:
    for m_f in m_fs:
      for est in ests:
        # create/train model
        RFmodel = RandomForestRegressor(n_estimators=est, max_depth=max_d, min_samples_leaf=min_s_l, max_features=m_f, n_jobs=-1, random_state=42)
        RFmodel.fit(X_train, y_train)
        MTR_obj = MTR(model=RFmodel, X_train = X_train, X_test=X_test, y_train=y_train, y_test=y_test, feature_names=f_n, target_names=t_n)
        avgRuleLengths = 0
        avgEstimators = []
        # get results
        for i in range(len(X_test)):
          error = calc_al_error(y_test[i], allowed_error)
          rule = MTR_obj.explain(X_test[i], error) # explain instance
          feature_limits = MTR_obj.getFeatureLimits()
          avgRuleLengths += len(feature_limits.keys())
          estimators = MTR_obj.getAmountOfReduction() # get estimators
          avgEstimators.append(estimators[0])
        finalLen = avgRuleLengths/len(X_test)
        rule_lengths.append((7-finalLen))
        if finalLen < min_len:
          min_len = finalLen
          best_par = {"max_d=":max_d, "min_s_l=":min_s_l, "m_f=": m_f, "est=": est, "rule length=": finalLen}
        print(counter, '/', total, "| max_depth=",max_d," | ", "min_sample_leaf=",min_s_l," | ", "max_f=", m_f," | ", "est=", est," | ", "reduction=", (7-finalLen)," | ", "estimators=", round(np.array(avgEstimators).mean(),3),"/",est)
        counter += 1

1 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 100  |  reduction= 2.709677419354839  |  estimators= 92.903 / 100
2 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 150  |  reduction= 2.161290322580645  |  estimators= 142.032 / 150
3 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 200  |  reduction= 1.903225806451613  |  estimators= 189.484 / 200
4 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 250  |  reduction= 1.903225806451613  |  estimators= 236.871 / 250
5 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 300  |  reduction= 2.0  |  estimators= 277.903 / 300
6 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 350  |  reduction= 1.903225806451613  |  estimators= 330.161 / 350
7 / 19 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 400  |  reduction= 1.645161290322581  |  estimators= 378.903 / 400
8 / 19 | max_depth= 2  |  min_sample_leaf= 10  | 

In [10]:
print(rule_lengths)

[2.709677419354839, 2.161290322580645, 1.903225806451613, 1.903225806451613, 2.0, 1.903225806451613, 1.645161290322581, 1.774193548387097, 1.67741935483871, 1.580645161290323, 1.580645161290323, 1.709677419354839, 1.709677419354839, 1.709677419354839, 1.451612903225806, 1.451612903225806, 1.451612903225806, 1.451612903225806, 1.451612903225806]


In [11]:

for r in rule_lengths:
  print(r)

2.709677419354839
2.161290322580645
1.903225806451613
1.903225806451613
2.0
1.903225806451613
1.645161290322581
1.774193548387097
1.67741935483871
1.580645161290323
1.580645161290323
1.709677419354839
1.709677419354839
1.709677419354839
1.451612903225806
1.451612903225806
1.451612903225806
1.451612903225806
1.451612903225806
